# Error propagation

## Tasks

Solve the tasks below.
For each task, give reasons for your solution by commenting in the notebook.
In conclusion, summarize your findings and contextualize them. What have you learned? Do the results make sense?

Your results will be examined for plagiarism. Please use your own plot styles, articulate your own thoughts, and present your own experimental approaches.

a) Propagate uncertainties for the following expressions using [SymPy](https://www.sympy.org) following the examples for [uncorrelated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation.ipynb?flush_cache=false) and [correlated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation_corr.ipynb?flush_cache=false) from the FP web page.

i) Find expressions for the absolute uncertainty $\sigma_z$ for $z = x + y$ and $z = x - y$ 

ii) Find expressions for the relative uncertainty $\sigma_z / z$ for $z = x \cdot y, \; z = x / y$ and $z = x^n y^n$

iii) The acceleration of gravity with a simple pendulum is given by the following formula:
$$g = 4  \pi^2 \frac{L}{T^2}$$
The relevant variables are the length $L$ of the pendulum and the period $T$ with the corresponding errors $\sigma_L$ and $\sigma_T$.

iv) The energy of a rotating object is given by:
$$E = \frac{1}{2} I \omega^2$$
The relevant variables are the angular velocity $\omega$ and the moment of interia $I$ with the corresponding errors $\sigma_\omega$ and $\sigma_I$.

b) The radius $r$ and the height $h$ of a cylinder have been measured to $r = 5$ cm and $h = 2$ cm. The uncertainty for both measurements is $\sigma = 0.1$ cm. Determine the volume of the cylinder and its uncertainty assuming (i) that both measurements are uncorrelated and (ii) that both measurements are fully correlated.

c) The scattering angle and the radial distance of a certain particle can be determined from a position measurement $(x,y)$ 
$$r = \sqrt{x^2 + y^2}, \quad \theta = \mathrm{atan2}(y, x)$$
You find more on the [atan2](https://en.wikipedia.org/wiki/Atan2) function on wikipedia. The position ($x$,$y$) is measured with the corresponding uncertainties $\sigma_x$ and $\sigma_y$. Write a python function that returns the covariance matrix $U$ of $r$ and $\theta$ for a given covariance matrix $V$ of $x$ and $y$. Determine $U$ under the assumption that $x$ and $y$ are uncorrelated. Hint: The formulas you need can be found in the script.


### Solution

In [2]:
from sympy import *
from IPython.display import display, Latex

# taking inspiration for how to error prop with sympy from the FPs error prop notebooks
# boils down to regular gaussian error prop but we dont have to do the derivitaves ourselves

from sympy import symbols, sqrt, diff, pi, latex, simplify

def error_prop(f, vars):
    """Uncorrelated error propagation."""
    sum_expr = 0
    for (var, sigma) in vars:
        sum_expr += diff(f, var)**2 * sigma**2
    return sqrt(sum_expr)

def error_prop_corr(f, vars, cov_matrix):
    """Correlated error propagation."""
    sum_expr = 0
    n = len(vars)
    for i in range(n):
        for j in range(n):
            sum_expr += diff(f, vars[i]) * diff(f, vars[j]) * cov_matrix[i][j]
    return sqrt(sum_expr)



a) Propagate uncertainties for the following expressions using [SymPy](https://www.sympy.org) following the examples for [uncorrelated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation.ipynb?flush_cache=false) and [correlated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation_corr.ipynb?flush_cache=false) from the FP web page.

i) Find expressions for the absolute uncertainty $\sigma_z$ for $z = x + y$ and $z = x - y$ 

In [5]:
# define symbols for in and output 

x, y, sigma_x, sigma_y = symbols('x y sigma_x sigma_y', positive=True)

z_plus = x + y
sigma_z_add = error_prop(z_plus, [(x, sigma_x), (y, sigma_y)])
display(Latex(f"For $z = x + y$: $$\\sigma_z = {latex(sigma_z_add)}$$"))

z_minus = x - y
sigma_z_sub = error_prop(z_minus, [(x, sigma_x), (y, sigma_y)])
display(Latex(f"For $z = x - y$: $$\\sigma_z = {latex(sigma_z_sub)}$$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Absolute error of a sum is the geometric mean of the summands absolute errors – just as we've been taught in beginner lab courses.

ii) Find expressions for the relative uncertainty $\sigma_z / z$ for $z = x \cdot y, \; z = x / y$ and $z = x^n y^n$

In [7]:
# same things as before bascially
z_mul = x * y
sigma_z_mul = error_prop(z_mul, [(x, sigma_x), (y, sigma_y)])
rel_mul = (sigma_z_mul / z_mul).simplify()
display(Latex(f"For $z = xy$: $$\\frac{{\\sigma_z}}{{z}} = {latex(rel_mul)}$$"))

z_div = x / y
sigma_z_div = error_prop(z_div, [(x, sigma_x), (y, sigma_y)])
rel_div = (sigma_z_div / z_div).simplify()
display(Latex(f"For $z = x/y$: $$\\frac{{\\sigma_z}}{{z}} = {latex(rel_div)}$$"))

n = symbols('n', positive=True)
z_pow = (x**n) * (y**n)
sigma_z_pow = error_prop(z_pow, [(x, sigma_x), (y, sigma_y)])
rel_pow = (sigma_z_pow / z_pow).simplify()
display(Latex(f"For $z = x^n y^n$: $$\\frac{{\\sigma_z}}{{z}} = {latex(rel_pow)}$$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

We don't really like how Sympy formats the end result. But pulling the /xy terms into the square root reveals what we already know – for multiplicative processes the relative error of the result is the geometric mean of the factors' relative errors.

iii) The acceleration of gravity with a simple pendulum is given by the following formula:
$$g = 4  \pi^2 \frac{L}{T^2}$$
The relevant variables are the length $L$ of the pendulum and the period $T$ with the corresponding errors $\sigma_L$ and $\sigma_T$.

In [8]:
# having such a function makes life quite easy – just define the symbols and lets go
L, T, sigma_L, sigma_T = symbols('L T sigma_L sigma_T', positive=True)
g = 4 * pi**2 * L / T**2
sigma_g = error_prop(g, [(L, sigma_L), (T, sigma_T)]).simplify()
display(Latex(f"$$\\sigma_g = {latex(sigma_g)}$$"))

<IPython.core.display.Latex object>

iv) The energy of a rotating object is given by:
$$E = \frac{1}{2} I \omega^2$$
The relevant variables are the angular velocity $\omega$ and the moment of interia $I$ with the corresponding errors $\sigma_\omega$ and $\sigma_I$.

In [10]:
# anda gain
I, omega, sigma_I, sigma_omega = symbols('I omega sigma_I sigma_omega', positive=True)
E = 0.5 * I * omega**2
sigma_E = error_prop(E, [(I, sigma_I), (omega, sigma_omega)]).simplify()
display(Latex(f"$$\\sigma_E = {latex(sigma_E)}$$"))

<IPython.core.display.Latex object>

b) The radius $r$ and the height $h$ of a cylinder have been measured to $r = 5$ cm and $h = 2$ cm. The uncertainty for both measurements is $\sigma = 0.1$ cm. Determine the volume of the cylinder and its uncertainty assuming (i) that both measurements are uncorrelated and (ii) that both measurements are fully correlated.

In [12]:
# now distinguish between correlated and uncorrelated and plug in some numbers.
r, h, sigma_r, sigma_h = symbols('r h sigma_r sigma_h', positive=True)
V = pi * r**2 * h

# unncorrelated, just as function from earlier
sigma_V_uncorr = error_prop(V, [(r, sigma_r), (h, sigma_h)])

# fully correlated, eg rho =1 with corr function
cov_matrix = [[sigma_r**2, 1*sigma_r*sigma_h], [1* sigma_r*sigma_h, sigma_h**2]]
sigma_V_corr = error_prop_corr(V, [r, h], cov_matrix)

# sub real values
subs_dict = {r: 5, h: 2, sigma_r: 0.1, sigma_h: 0.1}
V_central = V.subs(subs_dict).evalf()
result_uncorr = sigma_V_uncorr.subs(subs_dict).evalf()
result_corr = sigma_V_corr.subs(subs_dict).evalf()

display(Latex(f" Volume: $V = {latex(V)} = {V_central:.1f} \, \mathrm{{cm}}^3$"))
display(Latex(f"Uncorrelated: $\\sigma_V = {result_uncorr:.1f} \, \mathrm{{cm}}^3$"))
display(Latex(f"Fully correlated: $\\sigma_V = {result_corr:.1f} \, \mathrm{{cm}}^3$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

This shows that having correlated variables leads to a larger error. Conceptually this makes sense – an error/change in one variable would induce an error/change in the other thus being mistaken in one variable leads to having an error in the other variable as well.

c) The scattering angle and the radial distance of a certain particle can be determined from a position measurement $(x,y)$ 
$$r = \sqrt{x^2 + y^2}, \quad \theta = \mathrm{atan2}(y, x)$$
You find more on the [atan2](https://en.wikipedia.org/wiki/Atan2) function on wikipedia. The position ($x$,$y$) is measured with the corresponding uncertainties $\sigma_x$ and $\sigma_y$. Write a python function that returns the covariance matrix $U$ of $r$ and $\theta$ for a given covariance matrix $V$ of $x$ and $y$. Determine $U$ under the assumption that $x$ and $y$ are uncorrelated. Hint: The formulas you need can be found in the script.

In [27]:
from sympy import symbols, sqrt, atan2, Matrix, diff, simplify

# define symbols as usual
x, y, rho,sigma_x, sigma_y = symbols('x y rho sigma_x sigma_y', real=True)

# def transformation to polar coords
r = sqrt(x**2 + y**2)
theta = atan2(y, x) # sympy atan2 has y first for some reason

# index the symbols
f = [r, theta]
a = [x, y]

# calculate Jacobian matrix G
G = Matrix([[diff(fi, aj) for aj in a] for fi in f])

# write general to see formula covariance matrix V for x, y
V = Matrix([[sigma_x**2, rho * sigma_x * sigma_y],
            [rho * sigma_x * sigma_y, sigma_y**2]])

# covariance transformation function
def cov_trafo(V):
    U = simplify(G * V * G.T)
    return U

# Compute
U_general = cov_trafo(V)
print("Covariance matrix U (general):")
display(Latex(latex(U_general)))

# Now plug in rho = 0 for uncorrelated variables
V_uncorr = V.subs(rho, 0)
U_uncorr = cov_trafo(V_uncorr)
print("\nCovariance matrix U (uncorrelated x and y):")

display(Latex(latex(U_uncorr)))

print(latex(U_general))
print(latex(U_uncorr.doit()))

Covariance matrix U (general):


<IPython.core.display.Latex object>


Covariance matrix U (uncorrelated x and y):


<IPython.core.display.Latex object>

\left[\begin{matrix}\frac{\sigma_{x} x \left(\rho \sigma_{y} y + \sigma_{x} x\right) + \sigma_{y} y \left(\rho \sigma_{x} x + \sigma_{y} y\right)}{x^{2} + y^{2}} & \frac{- \sigma_{x} y \left(\rho \sigma_{y} y + \sigma_{x} x\right) + \sigma_{y} x \left(\rho \sigma_{x} x + \sigma_{y} y\right)}{\left(x^{2} + y^{2}\right)^{\frac{3}{2}}}\\\frac{\sigma_{x} x \left(\rho \sigma_{y} x - \sigma_{x} y\right) - \sigma_{y} y \left(\rho \sigma_{x} y - \sigma_{y} x\right)}{\left(x^{2} + y^{2}\right)^{\frac{3}{2}}} & \frac{- \sigma_{x} y \left(\rho \sigma_{y} x - \sigma_{x} y\right) - \sigma_{y} x \left(\rho \sigma_{x} y - \sigma_{y} x\right)}{\left(x^{2} + y^{2}\right)^{2}}\end{matrix}\right]
\left[\begin{matrix}\frac{\sigma_{x}^{2} x^{2} + \sigma_{y}^{2} y^{2}}{x^{2} + y^{2}} & \frac{x y \left(- \sigma_{x}^{2} + \sigma_{y}^{2}\right)}{\left(x^{2} + y^{2}\right)^{\frac{3}{2}}}\\\frac{x y \left(- \sigma_{x}^{2} + \sigma_{y}^{2}\right)}{\left(x^{2} + y^{2}\right)^{\frac{3}{2}}} & \frac{\sigma_{x}^{2} y

No clue why the notebook suddenly decided to not render latex anymore.

General case:

$$U_{general} = \left[\begin{matrix}\frac{\sigma_{x} x \left(\rho \sigma_{y} y + \sigma_{x} x\right) + \sigma_{y} y \left(\rho \sigma_{x} x + \sigma_{y} y\right)}{x^{2} + y^{2}} & \frac{- \sigma_{x} y \left(\rho \sigma_{y} y + \sigma_{x} x\right) + \sigma_{y} x \left(\rho \sigma_{x} x + \sigma_{y} y\right)}{\left(x^{2} + y^{2}\right)^{\frac{3}{2}}}\\\frac{\sigma_{x} x \left(\rho \sigma_{y} x - \sigma_{x} y\right) - \sigma_{y} y \left(\rho \sigma_{x} y - \sigma_{y} x\right)}{\left(x^{2} + y^{2}\right)^{\frac{3}{2}}} & \frac{- \sigma_{x} y \left(\rho \sigma_{y} x - \sigma_{x} y\right) - \sigma_{y} x \left(\rho \sigma_{x} y - \sigma_{y} x\right)}{\left(x^{2} + y^{2}\right)^{2}}\end{matrix}\right]$$

And uncorrelated case:

$$U_{uncorr} = \left[\begin{matrix}\frac{\sigma_{x}^{2} x^{2} + \sigma_{y}^{2} y^{2}}{x^{2} + y^{2}} & \frac{x y \left(- \sigma_{x}^{2} + \sigma_{y}^{2}\right)}{\left(x^{2} + y^{2}\right)^{\frac{3}{2}}}\\\frac{x y \left(- \sigma_{x}^{2} + \sigma_{y}^{2}\right)}{\left(x^{2} + y^{2}\right)^{\frac{3}{2}}} & \frac{\sigma_{x}^{2} y^{2} + \sigma_{y}^{2} x^{2}}{\left(x^{2} + y^{2}\right)^{2}}\end{matrix}\right]$$

And one can see how, when uncorrelated variables are subject to a coordinate transforms "mixing" them together (e.g. radius and theta both depend on x,y each) the resulting variables are correlated.